In [12]:
import torch
import torch.nn as nn
import torch.utils.data as data

from torch import Tensor
from torch.distributions import Bernoulli, Distribution, Independent, Normal
from torchvision.datasets import MNIST
from torchvision.transforms.functional import to_pil_image, to_tensor
from tqdm import tqdm

import zuko

_ = torch.random.manual_seed(0)

In [13]:

trainset = MNIST(root="~/data", download=True, train=True, transform=to_tensor)
trainloader = data.DataLoader(trainset, batch_size=256, shuffle=True)

In [14]:
x = [trainset[i][0] for i in range(16)]
x = torch.cat(x, dim=-1)

to_pil_image(x)

In [15]:

class ELBO(nn.Module):
    def __init__(
        self,
        encoder: zuko.lazy.LazyDistribution,
        decoder: zuko.lazy.LazyDistribution,
        prior: zuko.lazy.LazyDistribution,
    ):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.prior = prior

    def forward(self, x: Tensor) -> Tensor:
        q = self.encoder(x)
        z = q.rsample()

        return self.decoder(z).log_prob(x) + self.prior().log_prob(z) - q.log_prob(z)

In [16]:
class GaussianModel(zuko.lazy.LazyDistribution):
    def __init__(self, features: int, context: int, hidden: int = 1024):
        super().__init__()

        self.hyper = nn.Sequential(
            nn.Linear(context, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 2 * features),
        )

    def forward(self, c: Tensor) -> Distribution:
        phi = self.hyper(c)
        mu, log_sigma = phi.chunk(2, dim=-1)

        return Independent(Normal(mu, log_sigma.exp()), 1)


class BernoulliModel(zuko.lazy.LazyDistribution):
    def __init__(self, features: int, context: int, hidden: int = 1024):
        super().__init__()

        self.hyper = nn.Sequential(
            nn.Linear(context, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, features),
        )

    def forward(self, c: Tensor) -> Distribution:
        phi = self.hyper(c)
        rho = torch.sigmoid(phi)

        return Independent(Bernoulli(rho), 1)

In [75]:
n_features = 16
encoder = GaussianModel(n_features, 784, hidden=256)
decoder = BernoulliModel(784, n_features, hidden=256)

prior = zuko.flows.MAF(
    features=n_features,
    transforms=3,
    hidden_features=(256, 256),
)


print(decoder)
n_params = sum(p.numel() for p in decoder.parameters())
# print with thousands separator
print(f"{n_params:,}")
print("---")
print(encoder)
n_params = sum(p.numel() for p in encoder.parameters())
# print with thousands separator
print(f"{n_params:,}")
print("---")
print(prior)
n_params = sum(p.numel() for p in prior.parameters())
# print with thousands separator
print(f"{n_params:,}")
print("---")

BernoulliModel(
  (hyper): Sequential(
    (0): Linear(in_features=16, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=784, bias=True)
  )
)
271,632
---
GaussianModel(
  (hyper): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=32, bias=True)
  )
)
274,976
---
MAF(
  (transform): LazyComposedTransform(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicAffineTransform()
      (order): [0, 1, 2, 3, 4, ..., 11, 12, 13, 14, 15]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=16, out_features=256, bias=True)
        (1): ReLU()
        (2): MaskedLinear(in_features=256, out_features=256, bias=True)
        (3): ReLU()
        (4): MaskedLinear(in_features=256, out_features=32, bias

In [76]:
elbo = ELBO(encoder, decoder, prior).cuda()
optimizer = torch.optim.Adam(elbo.parameters(), lr=1e-3)

for epoch in (bar := tqdm(range(64))):
    losses = []

    for x, _ in trainloader:
        x = x.round().flatten(-3).cuda()
        loss = -elbo(x).mean()
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        losses.append(loss.detach())

    losses = torch.stack(losses)

    bar.set_postfix(loss=losses.mean().item())

100%|██████████| 64/64 [06:17<00:00,  5.90s/it, loss=72.7]


In [77]:
z = prior().sample((n_features+1,))
print(z)
print(z.shape)
x = decoder(z)
print(x)
sample = x.sample(torch.Size([1]))
print(torch.max(sample))
print(torch.min(sample))
print(sample.shape)
y = sample.mean(dim=1)
y = y.reshape(-1, 28)
print(torch.max(y))
print(torch.min(y))
print(y.shape)
z = y
#to_pil_image(y)
#print(torch.max(torch.tensor(x)))
#print(torch.min(x))
x = x.mean
print(x.shape)
x = x.reshape(-1, 28, 28)
y = x.movedim(0, 1).reshape(28, -1)
print(torch.max(y))
print(torch.min(y))
print(y.shape)

to_pil_image(x.movedim(0, 1).reshape(28, -1))

tensor([[ 2.0948e+00, -2.2203e-01, -9.5326e+00, -3.6423e+00,  3.6458e+00,
          2.2593e-01,  1.5169e+00,  7.4738e+00, -4.9242e+00,  4.2030e-01,
         -2.9945e+00, -2.3906e+00, -2.0135e+00, -2.6839e+00, -2.7722e+00,
         -4.5534e+00],
        [-2.8519e+00, -2.9411e+00, -1.4257e+00, -1.2589e+01, -6.7205e-02,
          4.7825e+00,  4.3651e+00, -5.5354e+00, -2.6279e+00,  1.7060e+00,
          4.9544e+00,  1.1541e+00,  6.0123e-01, -4.0677e+00,  2.2275e+00,
          2.1673e+00],
        [ 4.4487e+00,  6.9984e+00, -5.9975e+00,  1.1340e+01, -2.5034e+00,
         -5.0558e+00, -1.2098e+00,  4.1107e+00, -6.5148e+00,  3.1279e+00,
          4.5749e+00,  1.0275e+01, -9.4106e+00,  4.3427e-01,  1.7028e+00,
         -3.5465e+00],
        [-1.5128e+00, -3.7530e+00, -1.0134e+01, -1.8204e+00,  2.7784e+00,
          3.7442e+00,  6.1496e+00,  5.4763e+00, -9.7073e-01,  4.0887e+00,
         -2.8771e+00, -4.4852e+00,  2.4449e+00,  6.8069e-01, -1.9139e+00,
          1.1641e-02],
        [-2.9815e+00

In [78]:
z

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0588,
         0.0588, 0.0588, 0.0588, 0.1176, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [79]:
to_pil_image(z)